<a href="https://colab.research.google.com/github/AparnaJuhi/SENN_Project/blob/main/SENN_isear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
from ast import literal_eval
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Remove the rows which contains values ['No resposne']

In [5]:
import pandas as pd
df=pd.read_csv('isear.csv',header=None)
print(len(df))

7652


In [6]:
df.drop(df[df[1] == '[ No response.]'].index, inplace = True)
print(len(df))

7575


It will store the sentences that people has said which demonstrate their feelings

In [7]:
feel_arr=df[1]
feel_arr[0]

'[ On days when I feel close to my partner and other friends.  \nWhen I feel at peace with myself and also experience a close \ncontact with people whom I regard greatly.]'

Now, Each sentence in fee_arr has been tokenised into words. Like "hi i am here" is converted to ["hi","i","am","here"] and now stored in feel_arr

In [8]:
feel_arr=[word_tokenize(sent) for sent in feel_arr]
print(feel_arr[0])

['[', 'On', 'days', 'when', 'I', 'feel', 'close', 'to', 'my', 'partner', 'and', 'other', 'friends', '.', 'When', 'I', 'feel', 'at', 'peace', 'with', 'myself', 'and', 'also', 'experience', 'a', 'close', 'contact', 'with', 'people', 'whom', 'I', 'regard', 'greatly', '.', ']']


As the length of sentence vary, but input to LSTM will be a fixed length of sentence, so here I have chosed 100 words as my sentence length. So when, the sentence will contains more than 100 words, padd function will discard the words after 100th position. If the sentence will have less than 100 words, so rest of the array will be filled with 'pad' word.

In [9]:
def padd(arr):
    for i in range(100-len(arr)):
        arr.append('<pad>')
    return arr[:100]
for i in range(len(feel_arr)):
  feel_arr[i]=padd(feel_arr[i])

Initialising glove array which contains a large amounts of words and their corresponding vector representaion in 50 dimenisons.For each word in feel_arr we will have a glove vector of 50 dimensions.

In [11]:
vocab_f=open('glove.6B.50d.txt')
word_to_emb={}
for line in vocab_f:
  word_to_emb[line.split()[0]]=[float(i) for i in line.split()[1:]] 
word_to_emb['<pad>']=[0]*50
print(len(word_to_emb['happy']))

50


In [12]:
embedded_feel_arr=[] 
for each_sentence in feel_arr:
    embedded_feel_arr.append([])
    for word in each_sentence:
        if word.lower() in word_to_emb:
            embedded_feel_arr[-1].append(word_to_emb[word.lower()])
        else:
            embedded_feel_arr[-1].append([0]*50)
print(embedded_feel_arr[0])

[[-0.61201, 0.98226, 0.11539, 0.014623, 0.23873, -0.067035, 0.30632, -0.64742, -0.38517, -0.03691, 0.094788, 0.57631, -0.091557, -0.54825, 0.25255, -0.14759, 0.13023, 0.21658, -0.30623, 0.30028, -0.23471, -0.17927, 0.9518, 0.54258, 0.31172, -0.51038, -0.65223, -0.48858, 0.13486, -0.40132, 2.493, -0.38777, -0.26456, -0.49414, -0.3871, -0.20983, 0.82941, -0.46253, 0.39549, 0.014881, 0.79485, -0.79958, -0.16243, 0.013862, -0.53536, 0.52536, 0.019818, -0.16353, 0.30649, 0.81745], [0.30045, 0.25006, -0.16692, 0.1923, 0.026921, -0.079486, -0.91383, -0.1974, -0.053413, -0.40846, -0.26844, -0.28212, -0.5, 0.1221, 0.3903, 0.17797, -0.4429, -0.40478, -0.9505, -0.16897, 0.77793, 0.33525, 0.3346, -0.1754, -0.12017, -1.7861, 0.29241, 0.55933, 0.029982, -0.32417, 3.9297, 0.1088, -0.57335, -0.17842, 0.0041748, -0.16309, 0.45077, -0.16123, -0.17311, -0.087889, -0.089032, 0.062001, -0.19946, -0.38863, -0.18232, 0.060751, 0.098603, -0.07131, 0.23052, -0.51939], [0.62236, 0.1973, 0.0023326, -0.44924, 0.0

In [13]:
X=np.array(embedded_feel_arr)
print(np.shape(X))

(7575, 100, 50)


In [21]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
Y = onehotencoder.fit_transform(np.array(df[0]).reshape(-1,1)).toarray()
print(np.shape(Y))

(7575, 7)


(7575, 6)


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [23]:
def model1(X,Y,input_size1,input_size2,output_size):
  m=Sequential()
  m.add(Bidirectional(LSTM(50,input_shape=(input_size1,input_size2))))
  m.add(Dropout(0.2))
  m.add(Dense(50,activation='relu'))
  m.add(Dropout(0.2))
  m.add(Dense(output_size,activation='softmax'))
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=10)
  return m

In [24]:
m=model1(X_train,Y_train,100,50,7)

Epoch 1/10
166/166 [==============================] - 11s 64ms/step - loss: 1.8780 - accuracy: 0.2348
Epoch 2/10
166/166 [==============================] - 11s 63ms/step - loss: 1.6910 - accuracy: 0.3482
Epoch 3/10
166/166 [==============================] - 11s 64ms/step - loss: 1.6177 - accuracy: 0.3799
Epoch 4/10
166/166 [==============================] - 11s 64ms/step - loss: 1.5426 - accuracy: 0.4180
Epoch 5/10
166/166 [==============================] - 11s 64ms/step - loss: 1.5007 - accuracy: 0.4340
Epoch 6/10
166/166 [==============================] - 11s 64ms/step - loss: 1.4576 - accuracy: 0.4462
Epoch 7/10
166/166 [==============================] - 11s 65ms/step - loss: 1.4125 - accuracy: 0.4751
Epoch 8/10
166/166 [==============================] - 11s 64ms/step - loss: 1.3752 - accuracy: 0.4857
Epoch 9/10
166/166 [==============================] - 11s 64ms/step - loss: 1.3240 - accuracy: 0.5147
Epoch 10/10
166/166 [==============================] - 11s 64ms/step - loss: 1.278